In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import pickle

# 1. Leer el dataset final
df = pd.read_csv('data/archivo_unido_FINAL3.csv')

# Convertir la variable objetivo "Precio" a numérico y eliminar registros sin precio
df['Precio'] = pd.to_numeric(df['Precio'], errors='coerce')
df = df.dropna(subset=['Precio'])

# 2. Seleccionar la variable objetivo y las variables predictoras
y = df['Precio']
features = ['Marca', 'Modelo', 'Provincia', 'Año', 'Kilometraje', 
            'Transmisión', 'Dirección', 'Motor', 'Tracción', 'Color', 'Combustible']
X = df[features]

# 3. Aplicar Label Encoding a las columnas categóricas y guardar los encoders
categorical_cols = ['Marca', 'Modelo', 'Provincia', 'Transmisión', 'Dirección', 'Tracción', 'Color', 'Combustible']
X_encoded = X.copy()
encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X_encoded[col].astype(str))
    encoders[col] = le  # Guardamos el encoder para la columna

# Guardar los encoders en un archivo para usarlos en la predicción
with open('encoders/encoders.pkl', 'wb') as f:
    pickle.dump(encoders, f)

# 4. Dividir el dataset en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=89)

# 5. Definir el modelo y configurar la búsqueda de hiperparámetros
rf = RandomForestRegressor(random_state=82)
param_grid = {
    'n_estimators': [300, 500, 700],
    'max_depth': [10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

grid_search = GridSearchCV(estimator=rf, 
                           param_grid=param_grid, 
                           cv=5, 
                           scoring='r2', 
                           n_jobs=-1, 
                           verbose=1)

grid_search.fit(X_train, y_train)
print("")
print("========")
print("")
print("Mejores parámetros encontrados:", grid_search.best_params_)
print("")
print("========")
print("")
# Utilizar el mejor estimador encontrado
best_rf = grid_search.best_estimator_

# 6. Evaluar el modelo en el conjunto de prueba
y_pred = best_rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("")
print("========")
print("")
print("Evaluación del modelo Random Forest:")
print("MSE:", mse)
print("MAE:", mae)
print("R²:", r2)
print("")
print("========")
print("")
# 7. Guardar el modelo entrenado en un archivo
joblib.dump(best_rf, 'models/modelo_random_forest.pkl')
print("El modelo se ha guardado en 'models/modelo_random_forest.pkl'")


Fitting 5 folds for each of 324 candidates, totalling 1620 fits


c:\Users\SEBPE\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
221 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\SEBPE\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\SEBPE\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
  File "c:\Users\SEBPE\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\bas



Mejores parámetros encontrados: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500}




Evaluación del modelo Random Forest:
MSE: 24291602.5038049
MAE: 3208.6106905470288
R²: 0.7485992619912492


El modelo se ha guardado en 'models/modelo_random_forest.pkl'


In [8]:
#pip install scikit-learn

In [6]:
import pandas as pd
import joblib
from sklearn.preprocessing import LabelEncoder
import pickle

# 1. Definir la lista de características utilizadas en el entrenamiento
features = ['Marca', 'Modelo', 'Provincia', 'Año', 'Kilometraje', 
            'Transmisión', 'Dirección', 'Motor', 'Tracción', 'Color', 'Combustible']

# 2. Crear un registro nuevo (ejemplo). Ajusta los valores según corresponda.
nuevo_registro = {
    'Marca': ['TOYOTA'],
    'Modelo': ['FORTUNER'],
    'Provincia': ['GUAYAS'],
    'Año': [2011],
    'Kilometraje': [150000],
    'Transmisión': ['AUTOMATICA'],
    'Dirección': ['HIDRAULICA'],
    'Motor': [2700],
    'Tracción': ['4x4'],
    'Color': ['BLANCO'],
    'Combustible': ['GASOLINA']
}
df_nuevo = pd.DataFrame(nuevo_registro)

# 3. Cargar los encoders guardados y aplicarlos a las columnas categóricas
categorical_cols = ['Marca', 'Modelo', 'Provincia', 'Transmisión', 'Dirección', 'Tracción', 'Color', 'Combustible']

with open('encoders/encoders.pkl', 'rb') as f:
    encoders = pickle.load(f)

df_nuevo_encoded = df_nuevo.copy()
for col in categorical_cols:
    # Se utiliza el encoder guardado para transformar el registro nuevo
    df_nuevo_encoded[col] = encoders[col].transform(df_nuevo_encoded[col].astype(str))

# 4. Cargar el modelo entrenado
modelo_rf = joblib.load('models/modelo_random_forest.pkl')

# 5. Realizar la predicción asegurándose de que el orden de las columnas coincide con 'features'
precio_predicho = modelo_rf.predict(df_nuevo_encoded[features])
print("Precio predicho:", precio_predicho[0])


Precio predicho: 22377.314666666665


In [ ]:
#==========================================================================================
#MODELO XGBOOST
#==========================================================================================

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import pickle

# 1. Leer el dataset final
df = pd.read_csv('data/archivo_unido_FINAL3.csv')

# Convertir la variable objetivo "Precio" a numérico y eliminar registros sin precio
df['Precio'] = pd.to_numeric(df['Precio'], errors='coerce')
df = df.dropna(subset=['Precio'])

# 2. Seleccionar la variable objetivo y las variables predictoras
y = df['Precio']
features = ['Marca', 'Modelo', 'Provincia', 'Año', 'Kilometraje', 
            'Transmisión', 'Dirección', 'Motor', 'Tracción', 'Color', 'Combustible']
X = df[features]

# 3. Aplicar Label Encoding a las columnas categóricas y guardar los encoders
categorical_cols = ['Marca', 'Modelo', 'Provincia', 'Transmisión', 'Dirección', 'Tracción', 'Color', 'Combustible']
X_encoded = X.copy()
encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X_encoded[col].astype(str))
    encoders[col] = le  # Guardamos el encoder para la columna

# Guardar los encoders en un archivo para usarlos en la predicción
with open('encoders/encoders_xgboost.pkl', 'wb') as f:
    pickle.dump(encoders, f)

# 4. Dividir el dataset en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=89)

# 5. Definir el modelo XGBoost y configurar la búsqueda de hiperparámetros
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=82)

param_grid = {
    'n_estimators': [300, 500, 700],
    'max_depth': [5, 10, 15],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'gamma': [0, 0.1, 0.2],
}

grid_search = GridSearchCV(estimator=xgb_model, 
                           param_grid=param_grid, 
                           cv=5, 
                           scoring='r2', 
                           n_jobs=-1, 
                           verbose=1)

grid_search.fit(X_train, y_train)

print("\n========\n")
print("Mejores parámetros encontrados:", grid_search.best_params_)
print("\n========\n")

# Utilizar el mejor estimador encontrado
best_xgb = grid_search.best_estimator_

# 6. Evaluar el modelo en el conjunto de prueba
y_pred = best_xgb.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n========\n")
print("Evaluación del modelo XGBoost:")
print("MSE:", mse)
print("MAE:", mae)
print("R²:", r2)
print("\n========\n")

# 7. Guardar el modelo entrenado en un archivo
joblib.dump(best_xgb, 'models/modelo_xgboost.pkl')
print("El modelo se ha guardado en 'modelo_xgboost.pkl'")


Fitting 5 folds for each of 729 candidates, totalling 3645 fits


Mejores parámetros encontrados: {'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 700, 'subsample': 0.9}




Evaluación del modelo XGBoost:
MSE: 17756015.678265683
MAE: 2593.7936584943363
R²: 0.8162379182307241


El modelo se ha guardado en 'modelo_xgboost.pkl'


In [10]:
import pandas as pd
import joblib
from sklearn.preprocessing import LabelEncoder
import pickle

# 1. Definir la lista de características utilizadas en el entrenamiento
features = ['Marca', 'Modelo', 'Provincia', 'Año', 'Kilometraje', 
            'Transmisión', 'Dirección', 'Motor', 'Tracción', 'Color', 'Combustible']

# 2. Crear un registro nuevo (ejemplo)
nuevo_registro = {
    'Marca': ['TOYOTA'],
    'Modelo': ['FORTUNER'],
    'Provincia': ['GUAYAS'],
    'Año': [2011],
    'Kilometraje': [150000],
    'Transmisión': ['AUTOMATICA'],
    'Dirección': ['HIDRAULICA'],
    'Motor': [2700],
    'Tracción': ['4x4'],
    'Color': ['BLANCO'],
    'Combustible': ['GASOLINA']
}
nuevo_registro = {
    'Marca': ['CHEVROLET'],
    'Modelo': ['AVEO ACTIVO'],
    'Provincia': ['PICHINCHA'],
    'Año': [2011],
    'Kilometraje': [150000],
    'Transmisión': ['MANUAL'],
    'Dirección': ['MECANICA'],
    'Motor': [1600],
    'Tracción': ['4x2'],
    'Color': ['AMARILLO'],
    'Combustible': ['GASOLINA']
}
df_nuevo = pd.DataFrame(nuevo_registro)

# 3. Cargar los encoders guardados y aplicarlos a las columnas categóricas
categorical_cols = ['Marca', 'Modelo', 'Provincia', 'Transmisión', 'Dirección', 'Tracción', 'Color', 'Combustible']

with open('encoders/encoders_xgboost.pkl', 'rb') as f:
    encoders = pickle.load(f)

df_nuevo_encoded = df_nuevo.copy()
for col in categorical_cols:
    df_nuevo_encoded[col] = encoders[col].transform(df_nuevo_encoded[col].astype(str))

# 4. Cargar el modelo entrenado
modelo_xgb = joblib.load('models/modelo_xgboost.pkl')

# 5. Realizar la predicción asegurándose de que el orden de las columnas coincide con 'features'
precio_predicho = modelo_xgb.predict(df_nuevo_encoded[features])
print("Precio predicho:", precio_predicho[0])


Precio predicho: 8893.264


In [ ]:

#=======================22439.43==20299.37===19956.922===17580.81===========================================================
#MODELO CatBoostRegressor
#==========================================================================================

In [6]:
#!pip install catboost

In [5]:
#pip install --upgrade numpy

In [4]:

#pip install --upgrade catboost

In [2]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

# Cargar los datos
df = pd.read_csv('data/archivo_unido_FINAL3.csv')
df['Precio'] = pd.to_numeric(df['Precio'], errors='coerce')
df = df.dropna(subset=['Precio'])

# Variables
y = df['Precio']
features = ['Marca', 'Modelo', 'Provincia', 'Año', 'Kilometraje', 'Transmisión', 'Dirección', 'Motor', 'Tracción', 'Color', 'Combustible']
X = df[features]

# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=89)

# Definir modelo CatBoost
cat_model = CatBoostRegressor(iterations=500, depth=10, learning_rate=0.05, loss_function='RMSE', random_seed=82)

# Entrenar modelo (CatBoost maneja automáticamente variables categóricas)
cat_model.fit(X_train, y_train, cat_features=['Marca', 'Modelo', 'Provincia', 'Transmisión', 'Dirección', 'Tracción', 'Color', 'Combustible'], verbose=False)

# Evaluación
y_pred = cat_model.predict(X_test)
print("MSE:", mean_squared_error(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred))

# Guardar modelo
joblib.dump(cat_model, 'models/modelo_catboost.pkl')

MSE: 18816904.171669405
MAE: 2906.343191514214
R²: 0.8052584799600331


['models/modelo_catboost.pkl']

In [18]:
import pandas as pd
import joblib
from catboost import CatBoostRegressor
import pickle

# 1. Definir la lista de características utilizadas en el entrenamiento
features = ['Marca', 'Modelo', 'Provincia', 'Año', 'Kilometraje', 
            'Transmisión', 'Dirección', 'Motor', 'Tracción', 'Color', 'Combustible']

# 2. Crear un registro nuevo (ejemplo)
nuevo_registro = {
    'Marca': ['TOYOTA'],
    'Modelo': ['FORTUNER'],
    'Provincia': ['PICHINCHA'],
    'Año': [2011],
    'Kilometraje': [150000],
    'Transmisión': ['AUTOMATICA'],
    'Dirección': ['HIDRAULICA'],
    'Motor': [2700],
    'Tracción': ['4x4'],
    'Color': ['BLANCO'],
    'Combustible': ['GASOLINA']
}
df_nuevo = pd.DataFrame(nuevo_registro)

# 3. Cargar los encoders guardados y aplicar a las columnas categóricas
categorical_cols = ['Marca', 'Modelo', 'Provincia', 'Transmisión', 'Dirección', 'Tracción', 'Color', 'Combustible']

with open('encoders/encoders.pkl', 'rb') as f:
    encoders = pickle.load(f)

df_nuevo_encoded = df_nuevo.copy()
for col in categorical_cols:
    if col in encoders:
        df_nuevo_encoded[col] = encoders[col].transform(df_nuevo_encoded[col].astype(str))
    else:
        print(f"⚠️ Advertencia: No se encontró encoder para {col}")

# 4. Cargar el modelo entrenado
modelo_catboost = joblib.load('models/modelo_catboost.pkl')

# 5. Realizar la predicción asegurándose de que el orden de las columnas coincide con 'features'
precio_predicho = modelo_catboost.predict(df_nuevo_encoded[features])

print("🚗 Precio predicho del vehículo:", precio_predicho[0])


🚗 Precio predicho del vehículo: 18179.386456884837
